# Dynamic Refugee Matching
# Demonstration

This notebook demonstrates the fuctioning of the mechanism introduced by Andersson, Ehlers and Martinello (2018). All the necessary documentation, requirements and dependecies should be documented in the package. If you have any comment, spot any bug or some documentation is missing, please let us know.

This notebook replicates and illustrates the allocation example we provide in the paper (Tables 1 and 2). The purpose of this notebook is to convey the intuition behind our allocation mechanism, and to perform some simple benchmarking against naive allocation rules. Moreover, this notebook is meant to introduce you to the simulations we perform in the paper, which are replicable with a second notebook (``simulations.ipynb``).

In [1]:
import numpy as np
import scipy as sp
import pandas as pd 
pd.options.mode.chained_assignment = None
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

%load_ext autoreload
%autoreload 2

## Problem Description
In this section we replicate the allocation example provided in the paper. The problem we want to tackle is to allocate a flow of $N$ refugees to $M$ localities. Refugees arrive sequentially, and need to be processed (and assigned) as they arrive. 

As shown by Bansak et al. (2018) and Trapp et al. (2018), individual refugees can have different probabilities of integrating in different localities. We coarsen these probabilities into a binary indicator. That is, every refugee can be either *acceptable* or *non-acceptable* for a given locality. We indicate acceptability in the refugee flows by a N$\times$M matrix, where each row represents a refugee and each column a locality. This matrix will be populated by a binary variable (0/1) indicating acceptability, and denoting whether a locality-asylum seeker match is a good one or not. We call this matrix **scoring matrix**, or ``scores``.

Localities might be subject to capacity constraints, or might be of different sizes and thus subject to a quotas system. The algorithm can accomodate these constraints trough a 1$\times$M **quotas vector**, or ``quotas``. This vector is an array of integers, denoting the maximum number of refugees that can be assigned to each locality $m$.


## Sequential assignment (benchmark)
Throughout the paper we benchmark the performance of our assignment mechanism with that of a naive sequential assignment rule. Sequrntial assignment rules typically outperforms truly random assignments as in the absence of quotas they guarantee that the number of asylum seekers assigned to each locality differs by at most one.

To familiarize with scoring and assignment matrixes, we begin by providing an example of sequential assignment, with and without quotas. We define the **scoring matrix** as the transpose of Table 1 of the paper. This matrix lists asylum seekers $i\in{1,\ldots,N}$ by row, and localities by column. A locality-asylum seeker match $(i,j)$ is considered acceptable iff element $(i,j)$ of the coring matrix is equal to 1. So the first two seekers are unacceptable in all three localities, seekers three and four are acceptable in all localities, and seeker 8 is acceptable only in localities 2 and 3.

We can optionally define a quotas array denoting the maximum capacity of each locality. Here, locality 2 is the largest and can accomodate at most 6 refugees. The sum of capacities should be greater than or equal to the number of asylum seekers in the flow $N$.

In [6]:
from dynamic_refugee_matching.assignment import assign_seq, assign_random
# Input scoring matrix
scores_example = np.array(  [[0, 0, 0],
                             [0, 0, 0],
                             [1, 1, 1],
                             [1, 1, 1],
                             [0, 0, 0],
                             [0, 0, 0],
                             [1, 1, 0],
                             [0, 1, 1],
                             [0, 0, 0],
                             [1, 1, 1]])

quotas = np.array([2,6,3])

## Assign asylum seekers sequentially ##
seq = assign_seq(scores_example)
seq_quotas = assign_seq(scores_example, vector_quotas=quotas)

## Print assignments ##
print("Sequential assigment without quotas\n", seq.assignment)
print("Sequential assigment with quotas\n", seq_quotas.assignment)

Sequential assigment without quotas
 [[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]]
Sequential assigment with quotas
 [[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]]


The assigment matrix is of the same shape of the scoring matrix, $N\times M$. Each row sums to one, and cells (i,j) such that ``assignment[i,j]=1`` denote assignment of asylum seeker $i$ to locality $j$. The assignment rule here is easy to grasp. The algorithm assigns asylum seekers sequentially, starting from locality 1, until all capacities are filled. In fact, this algorithm does not take the contents of the scoring matrix under consideration at all (we only need to input it to measure $N$ and $M$). 

Note that in this particular example the assignment chosen by the naive sequential algorithm is nonetheless efficient, as all acceptable asylum seekers are assigned to a municipality that finds them acceptable. In other words, the sum of all successfully matched asylum seekers is equal to the number of demanded and over-demanded asylum seekers

In [4]:
from dynamic_refugee_matching.evaluate import evaluate_efficiency_case
evaluate_efficiency_case(scores_example, seq)

Sum of demanded and over-demanded asylum seekers : 5 
Sum of well-matched asylum seekers               : 5


## Mechanism with rotation (AEM)

We show here how our proposed mechanism would assign asylum seekers characterized by matrix ``scores_example``. This part of the notebook replicates Table 2 of the paper. Furthermore, we show how the mechanism would assign refugee in the presence of (binding) capacity constraints.

In [5]:
from dynamic_refugee_matching.assignment import assign
## Assign asylum seekers according to our rotation mechanism ##
aem = assign(scores_example)
aem_quotas = assign(scores_example, vector_quotas=quotas)

## Print assignments ##
print("AEM assigment without quotas\n", aem.assignment, "\n")
print("AEM assigment with quotas\n", aem_quotas.assignment, "\n")

## Evaluate assignment ##
evaluate_efficiency_case(scores_example, aem)

AEM assigment without quotas
 [[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]] 

AEM assigment with quotas
 [[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]] 

Sum of demanded and over-demanded asylum seekers : 5 
Sum of well-matched asylum seekers               : 5


Again (this time by construction), the final allocation is efficient. However, ``aem`` also guarantees fairness. 

In the paper we define fairness by the concept of envy. Theorem 1 in the paper shows that our mechanism satisfies envy bounded by a single asylum seeker. No envy means that no aggregation entity (locality), *while being in the market* (localities can exit the market with quotas), would like to exchange its assigned asylum seekers with those of **any other locality in the market**. With indivisible bundles, envy-free matching in general des not exist. However, our algorithm guarantees envy bounded by one: Envy between localities would disappear if they could obtain just one refugee from another bundle.

Note that as municipalities have heterogenous preferences envy is not symmetric. That is, in principle two localities can simultaneously envy each other, or not envy each other at all. Unlike standard approaches such as naive sequential assigments, our algorithm harnesses these heterogeneities to ensure fair matchings.

We characterize envy after $k$ arrivals with a $M\times M$ **envy matrix** $E_{k}$. Each pair $i,j$ indicates by how many asylum seekers locality $i$ (row) envies locality $j$ (column). The diagonal is naturally equal to zero. As Theorem 1 proves, in the assignment produced by our algorithm, no locality envies another by more than 1 asylum seekers (in this specific example, no locality envies any other by any number of asylum seekers).

In [5]:
from dynamic_refugee_matching.evaluate import calc_envy
print('Envy matrix after AEM assignment\n', calc_envy(aem.assignment, scores_example))

Envy matrix after AEM assignment
 [[ 0  0 -2]
 [ 0  0  0]
 [-2  0  0]]


However, the naive sequential assignment does not achieve a fair allocation. Specifically, by chance, the second locality envies the first by 3 asylum seekers. In other words, the second locality would need three more matches not to envy the first. Clearly, maintaining pareto-efficiency, AEM selects a superior allocation.

In [6]:
print('Envy matrix after sequential assignment\n', calc_envy(seq.assignment, scores_example))

Envy matrix after sequential assignment
 [[ 0 -5 -3]
 [ 3  0  0]
 [ 1  0  0]]


The performance of the the naive sequential assignment worsens with time, as the number of asylum seekers grows. To show this deterioration of performance, we duplicate the original asylum seeker flow by ten times, and compare the performance of our mechanism with those of a sequential assigment rule and random allocation.

With ten times more refugees, both the sequential and random allocation mechanisms are not pareto-efficient, as they result in fewer than the potential 50 appropriate matches in the simulated asylum seeker flows. Moreover, while no locality envies another after allocating 100 asylum seekers under the allocation selected by ``aem``, two out of three localities envy another with an alternative uninformed assignment.

In [7]:
from dynamic_refugee_matching.evaluate import characterize_assignments
# Set seed numpy
np.random.seed(1)
# Replicate original arrival flow x10
for i in range(10):
    if i == 0:
        scores_large = scores_example
    else:
        scores_large = np.vstack((scores_large,scores_example))

# Allocation quality summary
assignments = {
    'AEM':               assign(scores_example),
    'Sequential':        assign_seq(scores_example),
    'Random':            assign_random(scores_example),
    'AEM (10x)':         assign(scores_large),
    'Sequential (10x)':  assign_seq(scores_large),
    'Random (10x)':      assign_random(scores_large),
}

characterize_assignments(assignments)

,AEM,Sequential,Random,AEM (10x),Sequential (10x),Random (10x)
Sum of demanded and over-demanded asylum seekers,5,5,5,50,50,50
Sum of well-matched asylum seekers,5,5,5,50,44,42
# localities envying another by more than 1 AS,0,2,2,0,2,2
Maximum envy,0,3,1,0,3,12


## Dynamic implementation

``aem`` processes every arriving asylum seeker without extracting any information of future arrivals, but only using the envy from past allocations to guide its path. However, for simplicity so far we have used ordered flows of refugees, where all information on future arrivals in a flow is in principle available. 

We show here how ``aem`` can (and is meant to) be implemented at every single refugee arrival.

In practice, the algorithm only requires envy and past rankings at each assignment. With potentially binding quotas, ``vector_quotas`` also needs to be transferred across algorithm calls. All this necessary information, which we need to transfer across individual allocations, is contained in the object exported by ``aem``. We can thus implement ``aem`` dynamically and trasfer the necessary information across iterations (sylum seekers $k \in \{1,\ldots,N\}$ as follows:

In [8]:
from dynamic_refugee_matching.assignment import assign

# Dynamic implementation (running aem at every refugee arrival)
for k in range(scores_example.shape[0]):
    if k==0:
        aem_dy = assign(scores_example[:][k][np.newaxis], vector_quotas=np.array([10,10,10]))
        total_assignment = aem_dy.assignment
    else:
        aem_dy = assign(scores_example[:][k][np.newaxis], 
                        pi_init = aem_dy.pi, 
                        sigma_init = aem_dy.sigma, 
                        envy_init = aem_dy.envy_final,
                        vector_quotas = aem_dy.end_quotas
                       )
        total_assignment = np.vstack((total_assignment,aem_dy.assignment))
    
# Cross-check that the allocation is the same
print("Assignment with dynamic implementation:\n", total_assignment)
print("Assignment if all flow processed at once:  \n", total_assignment)


Assignment with dynamic implementation:
 [[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]]
Assignment if all flow processed at once:  
 [[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]]


## References

Andersson, T., L. Ehlers, and A. Martinello (2018). Dynamic Refugee Matching. Lund University Department of Economics Working Paper 2018: 7

Bansak, K., J. Ferwerda, J. Hainmueller, A. Dillon, D. Hangartner, and D. Lawrence (2018). Improving refugee integration through data-driven algorithmic assignment. Science 359, 325-329.

Trapp, Andrew C., Alexander Teytelboym, Alessandro Martinello, Tommy Andersson, and Narges Ahani. "Placement Optimization in Refugee Resettlement." Lund University Department of Economics Working Paper 2018: 23 (2018).